<a href="https://colab.research.google.com/github/eneaiten/Master-Statistics-Live-Series/blob/main/GXT_Parvo_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import numpy as np

file = "/content/ES_05_ES_05_0_20251105_1052.xlsx"
gxt_raw = pd.read_excel(file, header=None)

# Trova la riga header ("Time")
start_row = gxt_raw[
    gxt_raw.apply(lambda r: r.astype(str).str.contains("Time", case=False, na=False)).any(axis=1)
].index[0]

# Trova la prima riga numerica dopo l'header
def is_real_number(x):
    if pd.isna(x):
        return False
    try:
        float(x)
        return True
    except:
        return False

numeric_start = gxt_raw.iloc[start_row+1:, 0].apply(is_real_number).idxmax()

# Seleziona solo la parte dai dati numerici in poi
sub_gxt = gxt_raw.iloc[numeric_start:, :]

# Trova il primo NaN nella colonna 0
first_nan_idx = sub_gxt[sub_gxt.iloc[:, 0].isna()].index
if len(first_nan_idx) > 0:
    last_valid_idx = first_nan_idx[0] - 1
else:
    last_valid_idx = sub_gxt.index[-1]

# Taglio definitivo
clean = sub_gxt.loc[:last_valid_idx].copy()

# Inserisci header
headers = gxt_raw.iloc[start_row].tolist()
clean.columns = headers

#reset index
clean.reset_index(drop=True, inplace=True)

# Mostra tabella pulita
display(clean)
display(clean.info())


,TIME,VO2,VO2/kg,METS,VCO2,VE,RER,RR,Vt,FEO2,FECO2,HR,VE/,VE/,PECO2,PEO2,PetCO2,PetO2
0,0.290833,0.468499,9.863134,2.818038,0.331832,14.277988,0.708287,20.630371,0.692086,17.191559,2.868168,0,30.476034,43.027782,21.425217,128.420944,21.882301,127.804253
1,0.535167,0.507441,10.682963,3.052275,0.372404,15.253914,0.733886,24.556616,0.621173,17.118046,3.010833,0,30.060482,40.960701,22.490925,127.871803,22.711115,127.190353
2,0.755167,0.80651,16.979158,4.851188,0.569817,22.590494,0.706522,22.727272,0.993982,16.863176,3.109478,128,28.010183,39.645168,23.227802,125.967926,23.42889,125.440315
3,1.0065,1.148046,24.169384,6.905538,0.700345,25.680511,0.610032,23.872681,1.075728,15.944923,3.358962,128,22.368891,36.668362,25.091442,119.108582,25.431637,118.422394
4,1.288167,1.219119,25.66567,7.333049,0.734114,25.222071,0.602168,24.852072,1.014888,15.54896,3.582259,128,20.688763,34.357151,26.759472,116.150726,26.972235,115.674973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,16.501671,2.762413,58.156055,16.616016,2.959074,102.83638,1.071192,59.322033,1.733528,17.622953,3.540904,179,37.227016,34.752888,26.450554,131.643463,26.654383,130.311005
66,16.760008,2.743269,57.753033,16.500866,2.938122,102.946335,1.07103,58.064518,1.772965,17.649574,3.512403,179,37.526882,35.038139,26.23765,131.842316,26.400366,130.602249
67,17.00901,2.54882,53.659378,15.331251,2.75469,94.132782,1.080771,52.208832,1.803005,17.589846,3.600423,179,36.9319,34.171822,26.895157,131.396149,27.114388,130.638519
68,17.270346,2.266337,47.712357,13.632102,2.576471,84.392982,1.136844,45.918369,1.837892,17.578983,3.754331,166.5,37.237614,32.755268,28.044853,131.315002,28.259233,130.761368


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TIME      70 non-null     object
 1   VO2       70 non-null     object
 2   VO2/kg    70 non-null     object
 3   METS      70 non-null     object
 4   VCO2      70 non-null     object
 5   VE        70 non-null     object
 6   RER       70 non-null     object
 7   RR        70 non-null     object
 8   Vt        70 non-null     object
 9   FEO2      70 non-null     object
 10  FECO2     70 non-null     object
 11  HR        70 non-null     object
 12  VE/       70 non-null     object
 13  VE/       70 non-null     object
 14  PECO2     70 non-null     object
 15  PEO2      70 non-null     object
 16  PetCO2    70 non-null     object
 17  PetO2     70 non-null     object
dtypes: object(18)
memory usage: 10.0+ KB


None

In [49]:
# Parametri
points_per_step = 12   # es: ogni step ha 3 righe
last_points = 4        # quanti punti finali per step considerare
max_step = 5          # numero di step da analizzare (opzionale)

df = clean.copy()     # il DataFrame già pulito

#creo il contenitore vuoto con nome last_points_rows
last_points_rows = pd.DataFrame()

for step in range(max_step): #parte da 0 e va fino a 7
    # Indici dell'inizio e della fine dello step
    start_idx = step * points_per_step
    end_idx = (step + 1) * points_per_step - 1

    # Prendi gli ultimi 'last_points' righe dello step
    # Range(start, stop) -> genera tutti i numeri da start fino a stop -1
    # list() -> trasforma in lista per poterlo usare come lista di indici da passare a df.iloc
    last_rows_idx = list(range(end_idx - last_points + 1, end_idx + 1))

    # Controlla che gli indici siano validi
    last_rows_idx = [i for i in last_rows_idx if i < len(df)]

    # Aggiungi queste righe
    # pd.concat([x, y]) -> aggiungi dataset y a x verticalmente
    last_points_rows = pd.concat([last_points_rows, df.iloc[last_rows_idx]])

# Aggiungi colonna Step
# Vogliamo assegnare a ogni riga il numero dello step a cui appartiene
# i// si chiama floor division o divisione intera, serve a capire a quale step appartiene il punto

# Esempio numerico:
#points_per_step = 3
# Indici DataFrame: 0,1,2 → step 1 → 0//3=0, 1//3=0, 2//3=0
# Indici 3,4,5 → step 2 → 3//3=1, 4//3=1, 5//3=1
# +1 per far partire la numerazione degli step da 1 invece che da 0
last_points_rows['Step'] = [i // points_per_step + 1 for i in last_points_rows.index]

# Calcola media per step
step_means = last_points_rows.groupby('Step').mean().reset_index()
step_means.columns = (step_means.columns.str.strip().str.lower().str.replace(' ', '_')) # columns standardization, lower case + remove spaces
print(step_means.columns)
step_means.columns.values[13]="ve/vo2"
step_means.columns.values[14]="ve/vco2"
sub_step_means = step_means[['step','hr','vo2','vco2','ve', 've/vo2','ve/vco2']]
# print(sub_step_means.to_string(index=False)) # .to_string(index=False) -> remove the index column
display (sub_step_means)
sub_step_means.to_excel('sub_step_means_ES05.xlsx', index=False)

Index(['step', 'time', 'vo2', 'vo2/kg', 'mets', 'vco2', 've', 'rer', 'rr',
       'vt', 'feo2', 'feco2', 'hr', 've/', 've/', 'peco2', 'peo2', 'petco2',
       'peto2'],
      dtype='object')


,step,hr,vo2,vco2,ve,ve/vo2,ve/vco2
0,1,93.125,1.030571,0.740392,25.014785,24.271944,33.789707
1,2,107.375,1.4662,1.125322,35.395335,24.145514,31.455866
2,3,133.25,1.756188,1.459946,44.429743,25.308116,30.433419
3,4,158.0,2.110643,1.865635,57.058494,27.034261,30.582572
4,5,169.0,2.444189,2.320996,75.564098,30.912839,32.551321
